# Getting data using SQL

Most organizations today use relational databases to store all of part of its data in structured form. This makes important the ability to get data from these databases as an input for our data science workflow. Relational databases have adopted the SQL language as an standard, and its <a href="http://en.wikipedia.org/wiki/SQL:2011">last revision appeared in 2011</a>.

But today the SQL language is not only used on top of relational databases, but other storage technology also uses that. 

Examples are:
* Cloudera Impala: http://www.cloudera.com/content/cloudera/en/products-and-services/cdh/impala.html integrated via: https://pypi.python.org/pypi/impyla
* Hadoop HIVE: https://hive.apache.org/ integrated via: http://pythonhosted.org/PyHive/index.html

## Using SQL, the pythonic way

The data structures (tables, columns, keys) of relational databases do not match with the data structures used in programming languages. This makes some times cumbersome to deal with SQL and relational data from other languages (a.k.a. <i>the impedance mismatch</i>).

There are several ways of using SQL to get data from and put data in relational databases. <a href="http://www.sqlalchemy.org/">SQL Alchemy</a> is arguably the preferred library to fill the gap between Python and relational databases. 

SQL Alchemy is built on top of the Python <a href="http://legacy.python.org/dev/peps/pep-0249/">DBAPI</a>. DBAPI is the (somehow) official interface for relational databases in Pyhon, but when working with different database systems, it behaves in different ways. SQL Alchemy attempts to provide a more consistent and higher level interface. It is based on the following:
* Provide a higher level interface to DBAPI.
* But still allowing you to use the specifics of each database system.
* Allow for the use of Python-style declarative statements for common database tasks.
* Optionally, providing a Object-Relational-Mapping functionality. 

The last is useful as it allows you to save/retrieve Python objects directly in relational databases, but it is also sometimes controversial for performance reasons. It is more often used in Web development than in data science tasks, so we will not deal with it here.

## Connecting to a database

The first thing we need to do is to get an Engine object that represents "the database" in the sense of the database management system. 

In [15]:
from sqlalchemy import create_engine
db = create_engine('sqlite:///customers.db')
print type(db)

<class 'sqlalchemy.engine.base.Engine'>


In the above sentence we have created a database connection. The sqlite in the connection string indicates we are using SqLite as the database engine. Note that SQlite is not a full-fledged database manager but a lightweight engine. If using other sistems as PosgreSQL or MySQL, we would probably need to indicate some credentials in the connection string.

The <code>Engine</code> is actually more than a connection. It features some additional functionality as **conection pooling**, i.e. it opens several database connections when convenient and reuses them to maximize performance. It also deals with connexion closing when appropriate. It also does **auto-commit**, i.e. it commits transactions to the database behind the scenes.

<img src="http://docs.sqlalchemy.org/en/latest/_images/sqla_arch_small.png"/>

## Creating tables

If we don't have the tables in our database, we have to create them. The following is an example of a Table definition.

In [16]:
from sqlalchemy import MetaData, Table, Column, Integer, String
metadata = MetaData(db)
# Be sure that the databases does not exist previously:
users = Table('users', metadata,
             Column('user_id', Integer, primary_key=True),
             Column('name', String(40)),
             Column('age', Integer),
)
users.create()

We have created an empty table with three columns. As SQLite is really a file-based database, we should see now the file in the folder of this notebook.

## Inserting rows

For inserting rows we have to tell the table that inserts will follow.

In [17]:
db.echo = True # echo de SQL that is automatically generated.
i = users.insert()
i.execute(name='Peter', age=35)
i.execute({'name': 'John', 'age': 41},
          {'name': 'Matthew', 'age': 50},
          {'name': 'Luke', 'age': 65})
db.echo = False

2015-05-08 13:04:32,260 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, age) VALUES (?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (name, age) VALUES (?, ?)


2015-05-08 13:04:32,262 INFO sqlalchemy.engine.base.Engine ('Peter', 35)


INFO:sqlalchemy.engine.base.Engine:('Peter', 35)


2015-05-08 13:04:32,264 INFO sqlalchemy.engine.base.Engine COMMIT


...

INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (name, age) VALUES (?, ?)


2015-05-08 13:04:32,267 INFO sqlalchemy.engine.base.Engine (('John', 41), ('Matthew', 50), ('Luke', 65))


INFO:sqlalchemy.engine.base.Engine:(('John', 41), ('Matthew', 50), ('Luke', 65))


2015-05-08 13:04:32,269 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


Note that in the first call to execute, a single row is inserted, while we can insert multiple rows provided that we are passing a Python dictionary. Note also that we have not specified the user_id in each of the calls - the database will generate a value for us that is unique per row.

## Retrieving data

We can get the data available using SELECT statements. If we want to retrieve everything this is the way:

In [18]:
s = users.select()
db.echo = True
rs = s.execute()
db.echo = False

2015-05-08 13:04:32,282 INFO sqlalchemy.engine.base.Engine SELECT users.user_id, users.name, users.age 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.user_id, users.name, users.age 
FROM users


2015-05-08 13:04:32,283 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


Then the resultset returned by the execution of the SELECT statement can be iterated. 

In [19]:
rs.fetchone()

(1, u'Peter', 35)

Note it is retrieving a Python tuple. we can iterate the resultset with a for statement but we can also get a list of tuples.

In [20]:
rs.fetchall() # Note this will not get the row previously fetched.

[(2, u'John', 41), (3, u'Matthew', 50), (4, u'Luke', 65)]

## SQL clauses as Python expressions

In [21]:
s = users.select().where(users.c.age > 45)
rs = s.execute()
rs.fetchall()

[(3, u'Matthew', 50), (4, u'Luke', 65)]

## Using SQL directly

You can also use SQL directly.

In [22]:
rs = db.execute("select * from users")
for row in rs:
    print row

(1, u'Peter', 35)
(2, u'John', 41)
(3, u'Matthew', 50)
(4, u'Luke', 65)


And also use bounded parameters...

In [23]:
initial = "J%"
sql = "select name, age from users where name like :i"
rs = db.execute(sql, i=initial)
print rs.fetchone()["age"]

41


## Converting to pandas

As you can get the result of SQL statements, you now have the means to convert them. 

In [24]:
rs = db.execute("select * from users")
print type(rs)

# Now we have all in memory as a list of tuples.
import pandas as pd
col_name = []
col_age = []
for row in rs:
    col_name.append(row["name"])
    col_age.append(row["age"])
names = pd.Series(col_name)
ages = pd.Series(col_age)
mydf = pd.DataFrame({"Name":col_name, "Age":col_age})
mydf

<class 'sqlalchemy.engine.result.ResultProxy'>


,Age,Name
0,35,Peter
1,41,John
2,50,Matthew
3,65,Luke


## Converting to pandas DataFrame (directly)

In [25]:
df = pd.read_sql_query('SELECT * FROM users', db, index_col = 'user_id')
df.head(4)

,name,age
user_id,,
1,Peter,35
2,John,41
3,Matthew,50
4,Luke,65


In [26]:
print df["age"]

user_id
1          35
2          41
3          50
4          65
Name: age, dtype: int64
